In [59]:
import os
import json
import numpy as np
import cv2
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models
from PIL import Image
from sklearn.model_selection import train_test_split

In [60]:
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import StratifiedKFold

In [61]:
# Import OS libraries
import os
import itertools

# Data handling tools
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , classification_report


# Deep learning libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,  BatchNormalization, Activation, Dropout  
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam , Adamax
from tensorflow.keras import regularizers

# from googleapiclient.discovery import build
# from google.oauth2.credentials import Credentials
# from googleapiclient.http import MediaFileUpload

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")


# Check GPU availability
print("GPU is available" if tf.config.list_physical_devices('GPU') else "GPU is not available")

sns.set_style('whitegrid')

GPU is not available


In [62]:
# Diretório onde estão armazenadas as imagens sem ser separadas por pastas
image_dir = r'C:\Users\bruna\OneDrive - Universidade do Minho\Tese Mestrado em Bioinformática\AGAR_dataset\AGAR_dataset\dataset'

In [63]:
def load_image(image_path, target_size=(112, 112)):
    try:
        image = Image.open(image_path)
        image = image.resize(target_size)
        return np.array(image) / 255.0  # Normaliza a imagem para a faixa [0, 1]
    except Exception as e:
        print(f"Erro ao carregar a imagem {image_path}: {e}")
        return None

In [64]:
# Caminho para o arquivo de IDs das imagens com fundo dark
ids_dark = r'C:\Users\bruna\OneDrive - Universidade do Minho\Microbialdataset\dark_images_ids.txt'

# Carregar IDs do grupo de treinamento de um arquivo de texto
with open(ids_dark, 'r') as file:
    ids = [line.strip() for line in file.readlines()]

In [65]:
print(f"Número de IDs carregados: {len(ids)}")
print(f"Primeiros 5 IDs: {ids[:5]}")

Número de IDs carregados: 9649
Primeiros 5 IDs: ['10000', '10001', '10002', '10003', '10004']


In [66]:
# Dividir os IDs em treinamento e validação (por exemplo, 80% para treinamento e 20% para validação)
split_index = int(len(ids) * 0.8)
train_ids = ids[:split_index]
val_ids = ids[split_index:]

In [67]:
%%capture
# Listas para armazenar caminhos de imagem e rótulos correspondentes
train_data = []
val_data = []

# Iterar sobre todos os arquivos no diretório
for filename in os.listdir(image_dir):
    if filename.endswith('.json'):
        json_path = os.path.join(image_dir, filename)
        with open(json_path, 'r') as file:
            try:
                json_data = json.load(file)
            except json.JSONDecodeError as e:
                print(f"Erro ao carregar JSON {json_path}: {e}")
                continue

        sample_id = str(json_data.get('sample_id', None))
        if sample_id is None:
            print(f"ID da amostra ausente no JSON: {json_path}")
            continue

        image_filename = f"{sample_id}.jpg"
        image_path = os.path.join(image_dir, image_filename)
        if not os.path.exists(image_path):
            print(f"Arquivo de imagem não encontrado: {image_path}")
            continue

        classes = json_data.get('classes', [])
        if not classes:
            print(f"Chave 'classes' ausente ou vazia no JSON: {json_path}")
            continue

        data_tuple = (image_path, classes[0])
        if sample_id in train_ids:
            train_data.append(data_tuple)
        elif sample_id in val_ids:
            val_data.append(data_tuple)

In [68]:
print(f"Número de imagens no conjunto de treinamento: {len(train_data)}")
print(f"Número de imagens no conjunto de validação: {len(val_data)}")

Número de imagens no conjunto de treinamento: 7718
Número de imagens no conjunto de validação: 1930


In [69]:
# Separar caminhos de imagem e rótulos para treinamento e validação
train_images, train_labels = zip(*train_data)
val_images, val_labels = zip(*val_data)

In [70]:
# Converter os rótulos em formato adequado (numérico), se necessário
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(val_labels)


In [71]:
# Converter os rótulos em one-hot encoded
one_hot_encoder = OneHotEncoder(sparse=False)
train_labels_one_hot = one_hot_encoder.fit_transform(train_labels.reshape(-1, 1))
val_labels_one_hot = one_hot_encoder.transform(val_labels.reshape(-1, 1))

In [78]:
# Obter o número de classes a partir do one hot encoder
num_classes = len(one_hot_encoder.categories_[0])
num_classes

5

In [85]:
# Função para gerar batches de dados
def data_generator(image_paths, labels_one_hot, batch_size=8, target_size=(112, 112)):
    while True:
        for start in range(0, len(image_paths), batch_size):
            end = min(start + batch_size, len(image_paths))
            batch_paths = image_paths[start:end]
            batch_labels = labels_one_hot[start:end]
            batch_images = []
            for path in batch_paths:
                image = load_image(path, target_size)
                if image is not None:
                    batch_images.append(image)
            yield np.array(batch_images), np.array(batch_labels)

In [86]:
# Criar geradores de dados para treinamento e validação
train_generator = data_generator(train_images, train_labels_one_hot, batch_size=8, target_size=(112, 112))
val_generator = data_generator(val_images, val_labels_one_hot, batch_size=8, target_size=(112, 112))

In [87]:
# Criar o modelo da CNN
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(112, 112, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')  # Corrigido para usar `num_classes`
])

In [88]:
# Compilar o modelo
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Treine o modelo
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_images) // 8,  # Número total de batches
    epochs=20,
    validation_data=val_generator,
    validation_steps=len(val_images) // 8   # Número total de batches de validação
)

Epoch 1/20
964/964 [==============================] - 1695s 2s/step - loss: 1.5412 - accuracy: 0.2829 - val_loss: 1.5969 - val_accuracy: 0.2531
Epoch 2/20
964/964 [==============================] - 1785s 2s/step - loss: 1.4584 - accuracy: 0.3180 - val_loss: 1.6102 - val_accuracy: 0.2479
Epoch 3/20
964/964 [==============================] - 1751s 2s/step - loss: 1.3891 - accuracy: 0.3608 - val_loss: 1.4481 - val_accuracy: 0.3517
Epoch 4/20
964/964 [==============================] - 3262s 3s/step - loss: 1.3167 - accuracy: 0.4145 - val_loss: 1.4070 - val_accuracy: 0.4123
Epoch 5/20
964/964 [==============================] - 1820s 2s/step - loss: 1.1811 - accuracy: 0.5091 - val_loss: 1.2397 - val_accuracy: 0.5067
Epoch 6/20
964/964 [==============================] - 2103s 2s/step - loss: 1.0909 - accuracy: 0.5567 - val_loss: 1.2100 - val_accuracy: 0.5405
Epoch 7/20
964/964 [==============================] - 2053s 2s/step - loss: 1.0012 - accuracy: 0.6012 - val_loss: 1.1098 - val_accuracy: